# RAG

In [29]:
query="Who was Guiseppe Mazzini?"
# query="Covid 19"

## Environment setup

In [1]:
pip install langchain langchain-openai langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: lang

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

··········


## Ingestion

### Loading documents

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "jess301.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

print(len(docs))
docs

28


[Document(metadata={'source': 'jess301.pdf', 'page': 0}, page_content='EVENTS AND PROCESSES\nSECTION I\n2024-25\n'),
 Document(metadata={'source': 'jess301.pdf', 'page': 1}, page_content='2024-25\n'),
 Document(metadata={'source': 'jess301.pdf', 'page': 2}, page_content='3\nNationalism   in    Europe\nIn 1848, Frédéric Sorrieu, a French artist, prepared a series of four\nprints visualising his dream of a world made up of ‘democratic\nand social Republics’, as he called them. The first print (Fig. 1) of the\nseries, shows the peoples of Europe and America – men and women\nof all ages and social classes – marching in a long train, and offering\nhomage to the statue of Liberty as they pass by it. As you would\nrecall, artists of the time of the French Revolution personified Liberty\nas a female figure – here you can recognise the torch of Enlightenment\nshe bears in one hand and the Charter of the Rights of Man in the\nother. On the earth in the foreground of the image lie the shattered\n

### Splitting



In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

81

### Embeddings



In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

### Storing in vectorstore



In [11]:
pip install -qU langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00


In [12]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)

In [13]:
ids = vector_store.add_documents(documents=all_splits)

### Combined Ingestion

In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma



embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = Chroma(embedding_function=embeddings)

In [18]:
file_path = "jess305.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

ids = vector_store.add_documents(documents=all_splits)


## Retrieval

### Retrieving relevant documents

In [31]:
results = vector_store.similarity_search_with_relevance_scores(
    query, k=6, score_threshold=0.4
)
results

[(Document(metadata={'page': 11, 'source': 'jess301.pdf', 'start_index': 800}, page_content='One such individual was the Italian revolutionary Giuseppe\nMazzini. Born in Genoa in 1805, he became a member of the\nsecret society of the Carbonari. As a young man of 24, he was\nsent into exile in 1831 for attempting a revolution in Liguria.\nHe subsequently founded two more underground societies,\nfirst, Young Italy in Marseilles, and then, Young Europe in\nBerne, whose members were like-minded young men from\nPoland, France, Italy and the German states. Mazzini believed\nthat God had intended nations to be the natural units of\nmankind. So Italy could not continue to be a patchwork of\nsmall states and kingdoms. It had to be forged into a single\nunified republic within a wider alliance of nations. This\nunification alone could be the basis of Italian liberty. Following\nhis model, secret societies were set up in Germany, France,\nSwitzerland and Poland. Mazzini’s relentless opposition to

In [32]:
for i in range(len(results)):
    print(f"{i}: {results[i]}")
    print("--------------------------------")

0: (Document(metadata={'page': 11, 'source': 'jess301.pdf', 'start_index': 800}, page_content='One such individual was the Italian revolutionary Giuseppe\nMazzini. Born in Genoa in 1805, he became a member of the\nsecret society of the Carbonari. As a young man of 24, he was\nsent into exile in 1831 for attempting a revolution in Liguria.\nHe subsequently founded two more underground societies,\nfirst, Young Italy in Marseilles, and then, Young Europe in\nBerne, whose members were like-minded young men from\nPoland, France, Italy and the German states. Mazzini believed\nthat God had intended nations to be the natural units of\nmankind. So Italy could not continue to be a patchwork of\nsmall states and kingdoms. It had to be forged into a single\nunified republic within a wider alliance of nations. This\nunification alone could be the basis of Italian liberty. Following\nhis model, secret societies were set up in Germany, France,\nSwitzerland and Poland. Mazzini’s relentless opposition 

## Generation

In [33]:
context = "\n----------------\n".join([doc.page_content for doc, _ in results])
meta_data="\n----------------\n".join([str(doc.metadata) for doc, _ in results])

In [34]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate(template="""Answer the given query: {query} \nbased on given {context}.
Do not halucinate or generate answers based on pretrained data. If you can't find answer based on given context
simply say 'I don't know because I can't find any relevant context.'""")

prompt = prompt_template.invoke({"query": query, "context": context})
response = model.invoke(prompt)

print(response.content)
print(meta_data)

Giuseppe Mazzini was an Italian revolutionary born in Genoa in 1805. He became a member of the secret society of the Carbonari and was sent into exile in 1831 at the age of 24 for attempting a revolution in Liguria. Mazzini subsequently founded two more underground societies: Young Italy in Marseilles and Young Europe in Berne. He believed that nations were intended by God to be the natural units of mankind and advocated for the unification of Italy into a single republic as part of a wider alliance of nations. Mazzini's vision opposed monarchy and supported democratic republics, which frightened conservatives. Metternich described him as "the most dangerous enemy of our social order."
{'page': 11, 'source': 'jess301.pdf', 'start_index': 800}
----------------
{'page': 11, 'source': 'jess301.pdf', 'start_index': 1575}
----------------
{'page': 11, 'source': 'jess301.pdf', 'start_index': 0}


## Combined Retrieval and Generation

In [35]:
query="Tell me about Gutenberg press?"
from langchain_core.prompts import PromptTemplate

results = vector_store.similarity_search_with_relevance_scores(
    query, k=7, score_threshold=0.2
)

context = "\n----------------\n".join([doc.page_content for doc, _ in results])
meta_data="\n----------------\n".join([str(doc.metadata) for doc, _ in results])



prompt_template = PromptTemplate(template="""Answer the given query: {query} \nbased on given {context}.
Do not halucinate or generate answers based on pretrained data. If you can't find answer based on given context
simply say 'I don't know because I can't find any relevant context.'""")

prompt = prompt_template.invoke({"query": query, "context": context})
response = model.invoke(prompt)

print("\n\nRESPONSE:")
print(response.content)
print("\n\nREFERENCES:")
print(meta_data)



RESPONSE:
The Gutenberg press, developed by Johann Gutenberg in the 1430s, was an innovative printing technology that utilized movable type. Drawing inspiration from the olive press, Gutenberg designed a system where metal types were cast for each letter of the alphabet, allowing for the composition of different texts. By 1448, Gutenberg had perfected this system, and the first major book he printed was the Bible, producing around 180 copies over three years. This was considered fast for the time. The press could print 250 sheets on one side per hour. This technology initiated a print revolution, leading to a significant increase in book production across Europe, with 20 million copies by the end of the 15th century and 200 million by the 16th century. Initially, printed books closely resembled handwritten manuscripts, with spaces left for hand-drawn illustrations and decorations.


REFERENCES:
{'page': 6, 'source': 'jess305.pdf', 'start_index': 0}
----------------
{'page': 6, 'sourc